# Train CNN-based DQN with History Buffer
Largely inspired by the Atari paper by DeepMind, I want to build a deep Q network that uses CNN architecture to encode frames and feeds the fixed-length encoding to an RNN.

In [1]:
import gfootball.env as football_env
import matplotlib.pyplot as plt
import torch
from torch import nn
from collections import deque
import random
from tqdm.notebook import tqdm_notebook
import copy
import numpy as np

Import custom modules from src

In [2]:
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 2
import dqn_utils as dq
import training_ground as tg

#### History buffer
The `HistoryBuffer` class is already set up to handle the `(72, 96, 4)` pixel representation of the pitch. It does normalization and conversion to `float`. Picking an arbitrary number of frames to use for the agent's memory

In [3]:
history = dq.HistoryBuffer(10)

In [4]:
history.get_tensor().shape

torch.Size([10, 4, 96, 72])

#### Training strategy
The `TrainingPlan` class schedules a series of progressively more difficult training scenarios. We just tell it how many of each difficulty to use.

In [5]:
training = tg.TrainingPlan(basic_rounds=5,
                           easy_rounds=10,
                           medium_rounds=10,
                           hard_rounds=10,
                           full_match_rounds=10)

#### Architecture Specification
Now we want to set up the DQN architecture. Starting with an encoder which will be applied to each observation (i.e. the $n$ most recent frames). This is where the Atari model inspiration really begins.

* Inputs: $(n, C, L, W)$ tensor where $n$ is the history, $C$ the number of channels (4 here), and $L, W$ representing pitch dimensions
* Output: $(n, d)$ tensor where $d$ is the dimensionality of the encoding

This doesn't work super well for batched observations, but that's quick enough to handle downstream from this class

In [6]:
class CNNEncoder(nn.Module):
    def __init__(self, out_size):
        super(CNNEncoder, self).__init__()
        self.c1 = nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=1)
        self.c2 = nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1)
        self.c3 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)
        self.c4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.LeakyReLU()
        self.linear = nn.Linear(1536, out_size)
        
    def forward(self, x):
        h = self.relu(self.c1(x))
        h = self.relu(self.c2(h))
        h = self.relu(self.c3(h))
        h = self.relu(self.c4(h))
        flattened = h.flatten(-3)
        out = self.relu(self.linear(flattened))
        return out

In [7]:
enc = CNNEncoder(512)

In [8]:
enc(history.get_tensor()).shape

torch.Size([10, 512])

Now for the class which will use these encoders for each frame for each observation

In [9]:
class HistoryConvAgent(nn.Module):
    def __init__(self, dropout_p = 0.1):
        super(HistoryConvAgent, self).__init__()
        self.encoder = CNNEncoder(out_size=256)
        self.gru = nn.GRU(256, 256, num_layers=1, bidirectional=False, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 18)
        self.activation = nn.LeakyReLU()
        
    def forward(self, x):
        # batching doesn't play nicely here
        if x.ndim == 4:
            encoded = self.encoder(x)
            _, gru_out = self.gru(encoded.unsqueeze(0))
        else:
            encoded = torch.stack([self.encoder(x[i]) for i in range(x.shape[0])])
            _, gru_out = self.gru(encoded)
        gru_out = self.dropout(gru_out.squeeze())
        fc1_out = self.activation(self.fc1(gru_out))
        fc2_out = self.activation(self.fc2(fc1_out))
        return fc2_out
    
model = HistoryConvAgent()

Non-batched

In [10]:
model(history.get_tensor())

tensor([-3.6013e-04,  6.4678e-02,  5.2795e-02, -1.3404e-04,  2.1556e-02,
         2.8254e-02,  2.3506e-02,  4.7141e-02,  1.6004e-02, -1.1682e-04,
         4.5853e-02,  5.2699e-02, -3.2268e-05, -6.0494e-04,  6.6652e-02,
         4.0843e-02, -4.8987e-04,  2.5698e-03], grad_fn=<LeakyReluBackward0>)

Batched

In [11]:
model(torch.stack([history.get_tensor(), history.get_tensor() * 0.99])).shape

torch.Size([2, 18])

Awesome - how many params are we working with?

In [12]:
sum([np.prod(x.shape) for x in model.parameters()])

991986

Just under a cool 1M

### Training Loop

In [13]:
target_net = copy.deepcopy(model)

In [17]:
EPSILON = 0.2
EPS_DECAY = 0.999
GAMMA = 0.99
BATCH_SIZE = 32
BUFFER_SIZE = 256

loss_fn = nn.MSELoss()
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
replay_buffer = deque(maxlen=BUFFER_SIZE)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
epoch_losses = []
epoch_rewards = []
for match in tqdm_notebook(range(len(training.training_plan))):
    env = training.get_next()
    performance = dq.play_round_with_history(
        env,
        model=model, 
        target_network=target_net, 
        device=device, 
        loss_fn=loss_fn,
        optimizer=optim,
        sync_freq=10,
        replay_buffer=replay_buffer,
        history_length=10,
        batch_size=BATCH_SIZE,
        epsilon=EPSILON,
        gamma=GAMMA
    )
    epoch_rewards.append(performance['reward'])
    epoch_losses.append(np.mean(performance['losses']))
    EPSILON *= EPS_DECAY

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 